In [1]:
import pandas as pd
import glob
import numpy
import csv
from nltk.tokenize import word_tokenize
import xlrd
import re
from nltk import bigrams
from nltk import ngrams
import operator
from collections import Counter
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt

In [2]:
from random import randint
from numpy import array
from numpy import argmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [3]:
path = r'E:\Level 4 Project\dataset\final\preprocess' # use your path
all_files = glob.glob(path + "/*.csv")

In [4]:
li = []

In [5]:
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [6]:
df = pd.concat(li, axis=0, ignore_index=True, sort=False)

In [13]:
df.head()

,Date,Text,Text
0,2014-01-01 23:59:47+00:00,"Clearwater Beach,FL Scattered Thunderstorms t...",NaN
1,2014-01-01 23:57:47+00:00,Tourism disappeared because of Troubles: To...,NaN
2,2014-01-01 23:56:21+00:00,I'm at Department of Tourism and Hospitality ...,NaN
3,2014-01-01 23:56:03+00:00,'Like a Napa Valley wine tour': Pot tourism r...,NaN
4,2014-01-01 23:55:55+00:00,Focus : Recreation & Tourism Planning .,NaN


In [8]:
df.shape

(5492006, 3)

In [9]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

In [10]:
def preprocess_fornothash(s, lowercase=True):
    tokens = tokenize_fornothash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_fornothash(s):
    return tokens_re_fornothash.findall(s)


regex_str_fornothash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    #r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_fornothash = re.compile(r'(' + '|'.join(regex_str_fornothash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [11]:
def preprocess_forhash(s, lowercase=True):
    tokens = tokenize_forhash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_forhash(s):
    return tokens_re_forhash.findall(s)

regex_str_forhash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_forhash = re.compile(r'(' + '|'.join(regex_str_forhash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [12]:
punctuation = list(string.punctuation)
unnecssary = ['The','tourism','I','rt', 'via','...','â','click']
unnecssary_hash = ['#','#tourism']
ascii_words = ['\x92','\x96']
years = ['2018','2017','2016','2015','2014','2013']
numbers = ['nine']
stop = stopwords.words('english') + punctuation + unnecssary + ascii_words + years + unnecssary_hash + numbers

count_all = []
popular_common_words = []
i=0
sets=705

c =0
for x in range(sets):
    count_all.append(Counter())

for y in range(sets):
    for x in range(1, 1000):
        value = df.loc[i, "Text"]

        #terms_all = [term for term in preprocess_fornothash(value) if term not in stop]  
        terms_all = [term for term in preprocess_forhash(value)
                  if term.startswith('#') if term not in stop]
        i = i + 1
        count_all[y].update(terms_all)
    
    popular_common_words.append(count_all[y].most_common(10))
        

In [13]:
trendingTopicsAll = []

def processTrendingArray(str):
    global trendingTopicsAll
    b = [i for i,x in enumerate(trendingTopicsAll) if x == str]
    if(b == []):
        trendingTopicsAll.append(str)
        return len(trendingTopicsAll)-1
    else:
        return b[0]

In [14]:
trendingTopicsInEachDays = []
trendingIdsInEachDays = []
for index, y in enumerate(popular_common_words):
    trendingTopicsInEachDays.append([])
    trendingIdsInEachDays.append([])
    for x in y:
        trendingTopicsInEachDays[index].append(x[0])
        trendingIdsInEachDays[index].append(processTrendingArray(x[0]))
        

# print(trendingTopicsInEachDays)
# print("####################")
print(trendingTopicsAll)
# print("####################")
# print(trendingIdsInEachDays)

['#travel', '#trip', '#smtravel', '#news', '#photo', '#amsterdam', '#netherlands', '#holland', '#water', '#house', '#travelers', '#street', '#htmi', '#hospitality', '#money', '#life', '#colorado', '#marijuana', '#town', '#road', '#railway', '#path', '#pakistan', '#adventure', '#ecuador', '#home', '#old', '#infografia', '#infographic', '#ttot', '#thailand', '#vatican', '#googlenews', '#business', '#popefrancis', '#caribbean', '#amazing', '#chile', '#asmsg', '#ian1', '#ff', '#hhi', '#aglasem', '#jobs', '#jammu', '#spain', '#culture', '#iran', '#truetibet', '#cnn', '#carrefour', '#muscat', '#buy_omani_products', '#egypt', '#blog', '#photography', '#welcome_to_egypt', '#hotels', '#saveswedenswolves', '#backpacking', '#canal', '#haiti', '#southafrica', '#art', '#church', '#religion', '#qatar', '#africa', '#china', '#neworleanswill', '#food', '#cruisepeople', '#ofc14', '#rwanda', '#sell', '#keepwolveslisted', '#nysos14', '#expediachat', '#ratw', '#travelchatsa', '#unwto', '#revival', '#aiese

In [15]:
# generate a sequence of random numbers in [0, 99]
tweetIdsCount = -1
inputs = len(trendingTopicsAll)

def generate_sequence():
    global tweetIdsCount
    print("tweetIdsCount - ",tweetIdsCount)
    tweetIdsCount = tweetIdsCount+1
    return (trendingIdsInEachDays[tweetIdsCount])
    #return [23,12,45]
    
# one hot encode sequence
def one_hot_encode(sequence, n_unique=inputs):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# generate data for the lstm
def generate_data():
	# generate sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to 3d for input
	X = encoded.reshape(encoded.shape[0], 1, encoded.shape[1])
	return X, encoded    
    
# define model
model = Sequential() 
model.add(LSTM(50, input_shape=(1, inputs)))
model.add(Dense(inputs, activation='relu'))
model.add(Dense(inputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
for i in range(500):
	X, y = generate_data()
	model.fit(X, y, epochs=40, batch_size=1, verbose=2)
# evaluate model on new data
X, y = generate_data()
yhat = model.predict(X)

#print('Expected:  %s' % one_hot_decode(y))
#print('Predicted: %s' % one_hot_decode(yhat))

a = []
b = []
i = 0;
for x in range(10):
    if(one_hot_decode(y)[x] == one_hot_decode(yhat)[x]):
        i += 1
    a.append(trendingTopicsAll[one_hot_decode(y)[x]])
    b.append(trendingTopicsAll[one_hot_decode(yhat)[x]])
   
print("Expected: ",a)
print("Predicted: ",b)
print("Test Accuracy: ",(i/10)*100)

W0114 05:45:26.082185  8672 deprecation.py:506] From C:\Users\Mithila\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


tweetIdsCount -  -1


W0114 05:45:27.493582  8672 deprecation.py:323] From C:\Users\Mithila\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
10/10 - 2s - loss: 6.9871 - acc: 0.0000e+00
Epoch 2/40
10/10 - 0s - loss: 6.9220 - acc: 0.4000
Epoch 3/40
10/10 - 0s - loss: 6.8101 - acc: 0.5000
Epoch 4/40
10/10 - 0s - loss: 6.5752 - acc: 0.4000
Epoch 5/40
10/10 - 0s - loss: 6.0793 - acc: 0.2000
Epoch 6/40
10/10 - 0s - loss: 5.1221 - acc: 0.3000
Epoch 7/40
10/10 - 0s - loss: 3.5705 - acc: 0.2000
Epoch 8/40
10/10 - 0s - loss: 2.4608 - acc: 0.1000
Epoch 9/40
10/10 - 0s - loss: 2.2792 - acc: 0.2000
Epoch 10/40
10/10 - 0s - loss: 2.2439 - acc: 0.1000
Epoch 11/40
10/10 - 0s - loss: 2.2024 - acc: 0.1000
Epoch 12/40
10/10 - 0s - loss: 2.1765 - acc: 0.6000
Epoch 13/40
10/10 - 0s - loss: 2.1268 - acc: 0.6000
Epoch 14/40
10/10 - 0s - loss: 2.0693 - acc: 0.8000
Epoch 15/40
10/10 - 0s - loss: 2.0221 - acc: 0.9000
Epoch 16/40
10/10 - 0s - loss: 1.9773 - acc: 0.9000
Epoch 17/40
10/10 - 0s - loss: 1.9102 - acc: 1.0000
Epoch 18/40
10/10 - 0s - loss: 1.8246 - acc: 1.0000
Epoch 19/40
10/10 - 0s - loss: 1.7248 - acc: 1.0000
Epoch 20/40
10/10

10/10 - 0s - loss: 0.0120 - acc: 1.0000
Epoch 39/40
10/10 - 0s - loss: 0.0112 - acc: 1.0000
Epoch 40/40
10/10 - 0s - loss: 0.0105 - acc: 1.0000
tweetIdsCount -  3
Epoch 1/40
10/10 - 0s - loss: 2.8022 - acc: 0.8000
Epoch 2/40
10/10 - 0s - loss: 2.0614 - acc: 0.8000
Epoch 3/40
10/10 - 0s - loss: 1.3963 - acc: 0.8000
Epoch 4/40
10/10 - 0s - loss: 1.0766 - acc: 0.8000
Epoch 5/40
10/10 - 0s - loss: 0.9446 - acc: 0.8000
Epoch 6/40
10/10 - 0s - loss: 0.6802 - acc: 0.8000
Epoch 7/40
10/10 - 0s - loss: 0.4339 - acc: 0.9000
Epoch 8/40
10/10 - 0s - loss: 0.3321 - acc: 0.8000
Epoch 9/40
10/10 - 0s - loss: 0.2322 - acc: 1.0000
Epoch 10/40
10/10 - 0s - loss: 0.1965 - acc: 0.9000
Epoch 11/40
10/10 - 0s - loss: 0.1554 - acc: 0.9000
Epoch 12/40
10/10 - 0s - loss: 0.1679 - acc: 0.9000
Epoch 13/40
10/10 - 0s - loss: 0.1315 - acc: 0.9000
Epoch 14/40
10/10 - 0s - loss: 0.1156 - acc: 1.0000
Epoch 15/40
10/10 - 0s - loss: 0.0888 - acc: 1.0000
Epoch 16/40
10/10 - 0s - loss: 0.0710 - acc: 1.0000
Epoch 17/40
10

...

10/10 - 0s - loss: 1.3744e-05 - acc: 1.0000
Epoch 32/40
10/10 - 0s - loss: 1.3589e-05 - acc: 1.0000
Epoch 33/40
10/10 - 0s - loss: 1.3446e-05 - acc: 1.0000
Epoch 34/40
10/10 - 0s - loss: 1.3136e-05 - acc: 1.0000
Epoch 35/40
10/10 - 0s - loss: 1.3017e-05 - acc: 1.0000
Epoch 36/40
10/10 - 0s - loss: 1.2874e-05 - acc: 1.0000
Epoch 37/40
10/10 - 0s - loss: 1.2671e-05 - acc: 1.0000
Epoch 38/40
10/10 - 0s - loss: 1.2528e-05 - acc: 1.0000
Epoch 39/40
10/10 - 0s - loss: 1.2349e-05 - acc: 1.0000
Epoch 40/40
10/10 - 0s - loss: 1.2183e-05 - acc: 1.0000
tweetIdsCount -  483
Epoch 1/40
10/10 - 0s - loss: 3.1634 - acc: 0.8000
Epoch 2/40
10/10 - 0s - loss: 2.1713 - acc: 0.8000
Epoch 3/40
10/10 - 0s - loss: 1.3394 - acc: 0.8000
Epoch 4/40
10/10 - 0s - loss: 0.7267 - acc: 0.8000
Epoch 5/40
10/10 - 0s - loss: 0.1936 - acc: 1.0000
Epoch 6/40
10/10 - 0s - loss: 0.0744 - acc: 1.0000
Epoch 7/40
10/10 - 0s - loss: 0.0488 - acc: 1.0000
Epoch 8/40
10/10 - 0s - loss: 0.0205 - acc: 1.0000
Epoch 9/40
10/10 - 0s -

Epoch 21/40
10/10 - 0s - loss: 5.7169e-04 - acc: 1.0000
Epoch 22/40
10/10 - 1s - loss: 5.4236e-04 - acc: 1.0000
Epoch 23/40
10/10 - 0s - loss: 5.1772e-04 - acc: 1.0000
Epoch 24/40
10/10 - 0s - loss: 4.9851e-04 - acc: 1.0000
Epoch 25/40
10/10 - 0s - loss: 4.7137e-04 - acc: 1.0000
Epoch 26/40
10/10 - 0s - loss: 4.3645e-04 - acc: 1.0000
Epoch 27/40
10/10 - 0s - loss: 4.2233e-04 - acc: 1.0000
Epoch 28/40
10/10 - 0s - loss: 3.9781e-04 - acc: 1.0000
Epoch 29/40
10/10 - 0s - loss: 3.8172e-04 - acc: 1.0000
Epoch 30/40
10/10 - 0s - loss: 3.5552e-04 - acc: 1.0000
Epoch 31/40
10/10 - 0s - loss: 3.4806e-04 - acc: 1.0000
Epoch 32/40
10/10 - 0s - loss: 3.2548e-04 - acc: 1.0000
Epoch 33/40
10/10 - 0s - loss: 3.1146e-04 - acc: 1.0000
Epoch 34/40
10/10 - 0s - loss: 2.9661e-04 - acc: 1.0000
Epoch 35/40
10/10 - 0s - loss: 2.8652e-04 - acc: 1.0000
Epoch 36/40
10/10 - 0s - loss: 2.8116e-04 - acc: 1.0000
Epoch 37/40
10/10 - 0s - loss: 2.6002e-04 - acc: 1.0000
Epoch 38/40
10/10 - 0s - loss: 2.5158e-04 - acc:

Epoch 12/40
10/10 - 0s - loss: 0.0163 - acc: 1.0000
Epoch 13/40
10/10 - 0s - loss: 0.0134 - acc: 1.0000
Epoch 14/40
10/10 - 0s - loss: 0.0112 - acc: 1.0000
Epoch 15/40
10/10 - 0s - loss: 0.0087 - acc: 1.0000
Epoch 16/40
10/10 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 17/40
10/10 - 0s - loss: 0.0065 - acc: 1.0000
Epoch 18/40
10/10 - 0s - loss: 0.0056 - acc: 1.0000
Epoch 19/40
10/10 - 0s - loss: 0.0049 - acc: 1.0000
Epoch 20/40
10/10 - 0s - loss: 0.0045 - acc: 1.0000
Epoch 21/40
10/10 - 0s - loss: 0.0041 - acc: 1.0000
Epoch 22/40
10/10 - 0s - loss: 0.0036 - acc: 1.0000
Epoch 23/40
10/10 - 0s - loss: 0.0034 - acc: 1.0000
Epoch 24/40
10/10 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 25/40
10/10 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 26/40
10/10 - 0s - loss: 0.0026 - acc: 1.0000
Epoch 27/40
10/10 - 0s - loss: 0.0024 - acc: 1.0000
Epoch 28/40
10/10 - 0s - loss: 0.0022 - acc: 1.0000
Epoch 29/40
10/10 - 0s - loss: 0.0021 - acc: 1.0000
Epoch 30/40
10/10 - 0s - loss: 0.0019 - acc: 1.0000
Epoch 31/40


Epoch 2/40
10/10 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 3/40
10/10 - 0s - loss: 9.3219e-06 - acc: 1.0000
Epoch 4/40
10/10 - 0s - loss: 6.3299e-06 - acc: 1.0000
Epoch 5/40
10/10 - 0s - loss: 5.0425e-06 - acc: 1.0000
Epoch 6/40
10/10 - 0s - loss: 4.7206e-06 - acc: 1.0000
Epoch 7/40
10/10 - 0s - loss: 4.6014e-06 - acc: 1.0000
Epoch 8/40
10/10 - 0s - loss: 4.5418e-06 - acc: 1.0000
Epoch 9/40
10/10 - 0s - loss: 4.5180e-06 - acc: 1.0000
Epoch 10/40
10/10 - 0s - loss: 4.4465e-06 - acc: 1.0000
Epoch 11/40
10/10 - 0s - loss: 4.4226e-06 - acc: 1.0000
Epoch 12/40
10/10 - 0s - loss: 4.3630e-06 - acc: 1.0000
Epoch 13/40
10/10 - 0s - loss: 4.3630e-06 - acc: 1.0000
Epoch 14/40
10/10 - 0s - loss: 4.3153e-06 - acc: 1.0000
Epoch 15/40
10/10 - 0s - loss: 4.2915e-06 - acc: 1.0000
Epoch 16/40
10/10 - 0s - loss: 4.2557e-06 - acc: 1.0000
Epoch 17/40
10/10 - 0s - loss: 4.2438e-06 - acc: 1.0000
Epoch 18/40
10/10 - 0s - loss: 4.2080e-06 - acc: 1.0000
Epoch 19/40
10/10 - 0s - loss: 4.1842e-06 - acc: 1.0000
Epoc

10/10 - 0s - loss: 3.9458e-06 - acc: 1.0000
Epoch 30/40
10/10 - 0s - loss: 3.9338e-06 - acc: 1.0000
Epoch 31/40
10/10 - 0s - loss: 3.8742e-06 - acc: 1.0000
Epoch 32/40
10/10 - 1s - loss: 3.8027e-06 - acc: 1.0000
Epoch 33/40
10/10 - 0s - loss: 3.7670e-06 - acc: 1.0000
Epoch 34/40
10/10 - 0s - loss: 3.7074e-06 - acc: 1.0000
Epoch 35/40
10/10 - 0s - loss: 3.6358e-06 - acc: 1.0000
Epoch 36/40
10/10 - 0s - loss: 3.6001e-06 - acc: 1.0000
Epoch 37/40
10/10 - 1s - loss: 3.5762e-06 - acc: 1.0000
Epoch 38/40
10/10 - 0s - loss: 3.5166e-06 - acc: 1.0000
Epoch 39/40
10/10 - 0s - loss: 3.4570e-06 - acc: 1.0000
Epoch 40/40
10/10 - 0s - loss: 3.4332e-06 - acc: 1.0000
tweetIdsCount -  498
Epoch 1/40
10/10 - 0s - loss: 6.0584 - acc: 0.5000
Epoch 2/40
10/10 - 0s - loss: 3.2915 - acc: 0.6000
Epoch 3/40
10/10 - 0s - loss: 2.3137 - acc: 0.7000
Epoch 4/40
10/10 - 0s - loss: 1.6409 - acc: 0.7000
Epoch 5/40
10/10 - 0s - loss: 0.8047 - acc: 0.7000
Epoch 6/40
10/10 - 0s - loss: 0.3577 - acc: 0.9000
Epoch 7/40
10